## 準備
フォルダに、`gait.mp4`をアップロードしましょう。

## ライブラリ類のインポートなど

In [ ]:
import cv2
import numpy as np  # PythonのOpenCVでは、画像はnumpyのarrayとして管理される
from PIL import Image
import matplotlib.pyplot as plt
%matplotlib inline

def imshow(img):  # Colab用imshow関数として
  if img.ndim == 3:
    img = cv2.cvtColor(img,cv2.COLOR_BGR2RGB)
    display(Image.fromarray(img))
  else:
    display(Image.fromarray(img))

## フレーム間差分法の実装

`gait.mp4`は 8 fps、4秒間の動画（32フレーム）

`frame_target`を$t$、`frame_interval`を$i$と書くと、フレーム $t-i$, $t$, $t+i$ を使ってフレーム$t$の前景を推定している

In [ ]:
# 動画の読み込み
cap = cv2.VideoCapture('gait.mp4')

frame_target = 16  # 背景推定ターゲットのフレーム番号（gait.mp4の場合は、0～31）
frame_interval = 1 # フレーム間差分のインターバル。変更したければ変更

# 動画を画像列として格納
frames = []
while True:
    ret, frame = cap.read()
    if not ret:
        break
    frames.append(frame.astype(int))
cap.release()
# print(len(frames))

frame_prev = frame_target - frame_interval
frame_next = frame_target + frame_interval

if frame_prev < 0 or frame_next >= len(frames):
  print("error: Referencing outside video sequence")
else:
  # L2ノルム
  sub_prev = np.linalg.norm(frames[frame_target] - frames[frame_prev],axis=2).astype(np.uint8)
  sub_next = np.linalg.norm(frames[frame_next] - frames[frame_target],axis=2).astype(np.uint8)

  # 2値化
  thresh = 32
  _, binary_prev = cv2.threshold(sub_prev, thresh, 255, cv2.THRESH_BINARY)
  _, binary_next = cv2.threshold(sub_next, thresh, 255, cv2.THRESH_BINARY)

  # 論理積
  binary = np.stack([binary_prev, binary_next])
  binary = np.min(binary,axis=0)

  # 表示
  print("t-1, t")
  imshow(binary_prev)
  print("t, t+1")
  imshow(binary_next)

  print("Result")
  imshow(np.hstack([frames[frame_target].astype(np.uint8),cv2.cvtColor(binary, cv2.COLOR_GRAY2BGR)]))
